# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

E. Bañados  ->  E. Bañados  |  ['E. Bañados']
Y. Khusanova  ->  Y. Khusanova  |  ['Y. Khusanova']
I. J. M. Crossfield  ->  I. J. M. Crossfield  |  ['I. J. M. Crossfield']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
J. Li  ->  J. Li  |  ['J. Li']
F. Walter  ->  F. Walter  |  ['F. Walter']


Arxiv has 60 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2409.06772


extracting tarball to tmp_2409.06772...

 done.


E. Bañados  ->  E. Bañados  |  ['E. Bañados']
Y. Khusanova  ->  Y. Khusanova  |  ['Y. Khusanova']


...

/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_2694/1211882699.py:51: LatexWarning: 2409.07019 did not run properly
'PosixPath' object is not subscriptable
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


 done.


K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
J. Li  ->  J. Li  |  ['J. Li']
F. Walter  ->  F. Walter  |  ['F. Walter']


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:745: LatexWarning: Could not extract abstract from tmp_2409.07113/manuscript.tex
  warnings.warn(LatexWarning(f"Could not extract abstract from {self.main_file}"))


list index out of range


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.06772-b31b1b.svg)](https://arxiv.org/abs/2409.06772) | **Broad-Line AGN at $3.5<z<6$: The Black Hole Mass Function and a Connection with Little Red Dots**  |
|| A. J. Taylor, et al. -- incl., <mark>E. Bañados</mark>, <mark>Y. Khusanova</mark> |
|*Appeared on*| *2024-09-12*|
|*Comments*| *28 pages, 14 figures, 4 tables. Submitted to ApJ*|
|**Abstract**|            We present a sample of 50 H-alpha detected broad-line active galactic nuclei (BLAGN) at redshifts 3.5<z<6.8 using data from the CEERS and RUBIES surveys. We select these sources directly from JWST/NIRSpec G395M/F290LP spectra. We use a multi-step pre-selection and a Bayesian fitting procedure to ensure a high-quality sample of sources with broad Balmer lines and narrow forbidden lines. We compute rest-frame ultraviolet and optical spectral slopes for these objects, and determine that 10 BLAGN in our sample are also little red dots (LRDs). These LRD BLAGN, when examined in aggregate, show broader H-alpha line profiles and a higher fraction of broad-to-narrow component H-alpha emission than non-LRD BLAGN. Moreover, we find that ~66% of these objects are intrinsically reddened (beta (optical)>0), independent of the contributions of emission lines to the broadband photometry. We construct the black hole (BH) mass function at 3.5<z<6 after computing robust observational and line detection completeness corrections. This BH mass function shows broad agreement with both recent JWST/NIRSpec and JWST/NIRCam WFSS based BH mass functions, though we extend these earlier results to log(M(BH)/M(sun)) < 7. The derived BH mass function is consistent with a variety of theoretical models, indicating that the observed abundance of black holes in the early universe is not discrepant with physically-motivated predictions. The BH mass function shape resembles a largely featureless power-law, suggesting that any signature from black-hole seeding has been lost by redshift z~5-6. Finally, we compute the BLAGN UV luminosity function and find good agreement with JWST-detected BLAGN samples from recent works, finding that BLAGN hosts constitute <10% of the total observed UV luminosity at all but the brightest luminosities.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.07113-b31b1b.svg)](https://arxiv.org/abs/2409.07113) | **A Post-Starburst Pathway to Forming Massive Galaxies and Their Black Holes at z>6**  |
|| M. Onoue, et al. -- incl., <mark>K. Jahnke</mark>, <mark>J. Li</mark>, <mark>F. Walter</mark> |
|*Appeared on*| *2024-09-12*|
|*Comments*| *24 pages, 7 figures, submitted to a Nature journal*|
|**Abstract**|            Understanding the rapid formation of supermassive black holes (SMBHs) in the early universe requires an understanding of how stellar mass grows in the host galaxies. Here, we perform an analysis of rest-frame optical spectra and imaging from JWST of two quasar host galaxies at z>6 which exhibit Balmer absorption lines. These features in the stellar continuum indicate a lack of young stars, similar to low-redshift post-starburst galaxies whose star formation was recently quenched. We find that the stellar mass (log(M_* / M_sun) > 10.6) of each quasar host grew in a starburst episode at redshift 7 or 8. One of the targets exhibits little ongoing star formation, as evidenced by the photometric signature of the Balmer break and a lack of spatially resolved H-alpha emission, placing it well below the star formation main sequence at z = 6. The other galaxy is transitioning to a quiescent phase; together, the two galaxies represent the most distant massive post-starburst galaxies known. The maturity of these two galaxies is further supported by the stellar velocity dispersions of their host galaxies, placing them slightly above the upper end of the local M_BH - sigma_* relation. The properties of our two post-starburst galaxies, each hosting an active SMBH with log(M_BH / M_sun) > 9, suggests that black holes played a major role in shaping the formation of the first massive galaxies in the Universe.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.07019-b31b1b.svg)](https://arxiv.org/abs/2409.07019) | **The K2 and TESS Synergy III: search and rescue of the lost ephemeris for K2's first planet**  |
|| E. Thygesen, et al. -- incl., <mark>I. J. M. Crossfield</mark> |
|*Appeared on*| *2024-09-12*|
|*Comments*| *Accepted for publication in AJ. 15 pages, 6 figures, 8 tables*|
|**Abstract**|            K2-2 b/HIP 116454 b, the first exoplanet discovery by K2 during its Two-Wheeled Concept Engineering Test, is a sub-Neptune (2.5 $\pm$ 0.1 $R_\oplus$, 9.7 $\pm$ 1.2 $M_\oplus$) orbiting a relatively bright (KS = 8.03) K-dwarf on a 9.1 day period. Unfortunately, due to a spurious follow-up transit detection and ephemeris degradation, the transit ephemeris for this planet was lost. In this work, we recover and refine the transit ephemeris for K2-2 b, showing a $\sim40{\sigma}$ discrepancy from the discovery results. To accurately measure the transit ephemeris and update the parameters of the system, we jointly fit space-based photometric observations from NASA's K2, TESS, and Spitzer missions with new photometric observations from the ground, as well as radial velocities from HARPS-N that are corrected for stellar activity using a new modeling technique. Ephemerides becoming lost or significantly degraded, as is the case for most transiting planets, highlights the importance of systematically updating transit ephemerides with upcoming large efforts expected to characterize hundreds of exoplanet atmospheres. K2-2 b sits at the high-mass peak of the known radius valley for sub-Neptunes, and is now well-suited for transmission spectroscopy with current and future facilities. Our updated transit ephemeris will ensure no more than a 13-minute uncertainty through 2030.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2409.06772/./Figures/bhmf.png', 'tmp_2409.06772/./Figures/optvopt.png', 'tmp_2409.06772/./Figures/Field_CEERS.png', 'tmp_2409.06772/./Figures/Field_PRIMER.png']
copying  tmp_2409.06772/./Figures/bhmf.png to _build/html/
copying  tmp_2409.06772/./Figures/optvopt.png to _build/html/
copying  tmp_2409.06772/./Figures/Field_CEERS.png to _build/html/
copying  tmp_2409.06772/./Figures/Field_PRIMER.png to _build/html/
exported in  _build/html/2409.06772.md
    + _build/html/tmp_2409.06772/./Figures/bhmf.png
    + _build/html/tmp_2409.06772/./Figures/optvopt.png
    + _build/html/tmp_2409.06772/./Figures/Field_CEERS.png
    + _build/html/tmp_2409.06772/./Figures/Field_PRIMER.png
found figures ['tmp_2409.07113/./fig3_nat_v5.png', 'tmp_2409.07113/./fig1_nat1_v5.png', 'tmp_2409.07113/./fig1_nat2_v5.png', 'tmp_2409.07113/./fig4_nat_v3.png']
copying  tmp_2409.07113/./fig3_nat_v5.png to _build/html/
copying  tmp_2409.07113/./fig1_nat1_v5.png to _build/html/
copying  tmp_2409.0711

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand{\per}{\ensuremath{^{-1}}\xspace}$
$\newcommand{\Ha}{H\ensuremath{\alpha}\xspace}$</div>



<div id="title">

# Broad-Line AGN at $\mathbf{3.5<z<6}$:\ The Black Hole Mass Function and a Connection with Little Red Dots

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.06772-b31b1b.svg)](https://arxiv.org/abs/2409.06772)<mark>Appeared on: 2024-09-12</mark> -  _28 pages, 14 figures, 4 tables. Submitted to ApJ_

</div>
<div id="authors">

A. J. Taylor, et al. -- incl., <mark>E. Bañados</mark>, <mark>Y. Khusanova</mark>

</div>
<div id="abstract">

**Abstract:** We present a sample of 50 H $\alpha$ detected broad-line active galactic nuclei (BLAGN) at redshifts $3.5<z<6.8$ using data from the CEERS and RUBIES surveys. We select these sources directly from _JWST_ /NIRSpec G395M/F290LP spectra. We use a multi-step pre-selection and a Bayesian fitting procedure to ensure a high-quality sample of sources with broad Balmer lines and narrow forbidden lines. We compute rest-frame ultraviolet and optical spectral slopes for these objects, and determine that 10 BLAGN in our sample are also little red dots (LRDs). These LRD BLAGN, when examined in aggregate, show broader $\Ha$ line profiles and a higher fraction of broad-to-narrow component $\Ha$ emission than non-LRD BLAGN. Moreover, we find that $\sim66\%$ of these objects are intrinsically reddened ( $\beta_{opt}>0$ ), independent of the contributions of emission lines to the broadband photometry. We construct the black hole (BH) mass function at $3.5<z<6$ after computing robust observational and line detection completeness corrections. This BH mass function shows broad agreement with both recent _JWST_ /NIRSpec and _JWST_ /NIRCam WFSS based BH mass functions, though we extend these earlier results to log (M $_{BH}/M_{\odot}) < 7$ . The derived BH mass function is consistent with a variety of theoretical models, indicating that the observed abundance of black holes in the early universe is not discrepant with physically-motivated predictions.  The BH mass function shape resembles a largely featureless power-law, suggesting that any signature from black-hole seeding has been lost by redshift $z\sim$ 5--6.Finally, we compute the BLAGN UV luminosity function and find good agreement with _JWST_ -detected BLAGN samples from recent works, finding that BLAGN hosts constitute $\lesssim$ 10 \% of the total observed UV luminosity at all but the brightest luminosities.

</div>

<div id="div_fig1">

<img src="tmp_2409.06772/./Figures/bhmf.png" alt="Fig12" width="100%"/>

**Figure 12. -** The BLAGN BH mass function for $3.5<z<6$. Blue points indicate the fully completeness-corrected BH mass function from this work, and upward/downward pointing grey triangles indicate our BH mass function with only the line detection/observational correction applied. We mark our lightest mass point with a hollow blue point to indicate its low line detection completeness of $<20\%$. Grey squares indicate our uncorrected BH mass function. All of our error bars are Poissonian (following the  ([ and Gehrels 1986]())  prescription). Our partially corrected or uncorrected BH mass functions share the same error bars (in logarithmic space) as our fully corrected BH mass function and are omitted to reduce visual clutter. Red diamonds show the BH mass function derived from NIRCam WFSS data in [Matthee, Naidu and Brammer (2024)](). Note that [Matthee, Naidu and Brammer (2024)]() acknowledge that their lightest mass point (marked with a hollow red diamond) likely suffers from incompleteness and does not indicate a turnover of the BH mass function at $M_{\textrm{BH}}\sim10^7 M_{\odot}$. Yellow hexagons show the BH mass function at $3.5<z<4.25$ derived from ground-based SDSS+HSC data in [He, Akiyama and Enoki (2024)](). Green squares show the BH mass function at $z\sim6$ from [Wu, Shen and Jiang (2022)](), and pink x's show the BH mass function from [Shen and Kelly (2012)](), both derived from SDSS data. We show a Schecter fit to our data from $10^{6.5}<M_{\textrm{BH}}>10^{8.5}$ and the [He, Akiyama and Enoki (2024)]() data at $M_{\textrm{BH}}>10^{8.5} M_{\odot}$ as the dashed blue curve. Our corrected points show excellent agreement with the [Matthee, Naidu and Brammer (2024)]() points indicating that our observational completeness correction is performing well, and our overall BH mass function seems to extend the [He, Akiyama and Enoki (2024)]() BH mass function to lower masses. (*fig:BHmassfunction*)

</div>
<div id="div_fig2">

<img src="tmp_2409.06772/./Figures/optvopt.png" alt="Fig6" width="100%"/>

**Figure 6. -** Photometrically measured $\beta_{opt}$ versus Spectroscopically measured $\beta_{opt}$. We show the BLAGN in our sample that are identified as LRDs using the   ([Kocevski, Finkelstein and Barro 2024]())  criteria as red points, and  BLAGN in our sample that do not satisfy the  ([Kocevski, Finkelstein and Barro 2024]())  selection as blue points. Note that not all of our BLAGN have PRISM data, thus some are excluded from this figure. The black dashed line is the 1:1 line. Objects above this line show a redder $\beta_{opt}$ when measured from photometry instead of spectroscopy likely due to emission line boosting. However, despite the emission line boosting effects, the LRDs still exhibit red to moderately red $\beta_{opt}$, implying that they are intrinsically dust-reddened. (*fig:optvopt*)

</div>
<div id="div_fig3">

<img src="tmp_2409.06772/./Figures/Field_CEERS.png" alt="Fig4.1" width="50%"/><img src="tmp_2409.06772/./Figures/Field_PRIMER.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** Upper panel: the positions of the BLAGN sample (green circles), the CEERS NIRCam coverage (blue shading), and CEERS/RUBIES NIRSpec MSA footprints (black/red outlines). Lower panel: the same as the top panel, for the PRIMER-UDS field. Note that the sky areas shown in the two panels are not exactly to scale with one another. The BLAGN are broadly distributed over the observed areas of the CEERS and PRIMER-UDS fields. (*fig:fields*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.06772"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\araa}{Annu. Rev. Astron. Astrophys.}$
$\newcommand{\aar}{Astron. Astrophys. Rev.}$
$\newcommand{\aj}{Astron. J.}$
$\newcommand{\apj}{Astrophys. J.}$
$\newcommand{\apjl}{Astrophys. J. Lett.}$
$\newcommand{\apjs}{Astrophys. J. Suppl. Ser.}$
$\newcommand{\apss}{Astrophys. Space Sci.}$
$\newcommand{\aap}{Astron. Astrophys.}$
$\newcommand{\aapr}{Astron. Astrophys. Rev.}$
$\newcommand{\aaps}{Astron. Astrophys. Suppl.}$
$\newcommand{\mnras}{Mon. Not. R. Astron. Soc.}$
$\newcommand{\nar}{New Astronomy Reviews}$
$\newcommand{\nat}{Nature}$
$\newcommand{\nastro}{Nat. Astron.}$
$\newcommand{\ncomms}{Nat. Commun.}$
$\newcommand{\pasj}{Publ. Astron. Soc. Jpn}$
$\newcommand{\pasp}{Publ. Astron. Soc. Pacific}$
$\newcommand{\sci}{Science}$
$\newcommand{\arcsec}{\mbox{\ensuremath{^{\prime\prime}}}}$
$\newcommand{\farcs}{\mbox{\ensuremath{.\!\!^{\prime\prime}}}}$
$\newcommand{\etal}{et al.}$
$\newcommand{\hbeta}{H{\beta}}$
$\newcommand{\halpha}{H{\alpha}}$
$\newcommand{\hst}{{\it HST}}$
$\newcommand{\jw}{{\it JWST}}$
$\newcommand{\galight}{\textsf{galight}}$
$\newcommand{\sersic}{Sérsic}$
$\newcommand{\reff}{R_{\rm eff}}$
$\newcommand{\av}{A_{\rm v}}$
$\newcommand{\red}[1]{\textcolor{red}{{#1}}}$
$\newcommand{\blue}[1]{\textcolor{blue}{#1}}$
$\newcommand{\todo}[1]{\textcolor{blue}{\textbf{TODO: #1}}}$
$\newcommand{\note}[1]{\textcolor{cyan}{\textbf{NOTE: #1}}}$
$\newcommand{\lt}{\ensuremath <}$
$\newcommand{\gt}{\ensuremath >}$
$\newcommand{\targetone}{J2236+0032}$
$\newcommand{\targettwo}{J1512+4422}$
$\newcommand{\@maketitle}{$
${$
$\thispagestyle{empty}$
$\vskip-36pt$
${\raggedright\sffamily\bfseries\fontsize{20}{25}\selectfont \@title\par}$
$\vskip10pt$
${\raggedright\sffamily\fontsize{12}{16}\selectfont  \@author\par}$
$\vskip25pt$
$}$
$}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\mbh}{M_{\rm BH}}$
$\newcommand{\smass}{M_{*}}$</div>



<div id="title">

# A Post-Starburst Pathway to Forming Massive Galaxies and Their Black Holes at $\mathbf{z>6}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.07113-b31b1b.svg)](https://arxiv.org/abs/2409.07113)<mark>Appeared on: 2024-09-12</mark> -  _24 pages, 7 figures, submitted to a Nature journal_

</div>
<div id="authors">

M. Onoue, et al. -- incl., <mark>K. Jahnke</mark>, <mark>J. Li</mark>, <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:**            Understanding the rapid formation of supermassive black holes (SMBHs) in the early universe requires an understanding of how stellar mass grows in the host galaxies. Here, we perform an analysis of rest-frame optical spectra and imaging from JWST of two quasar host galaxies at z>6 which exhibit Balmer absorption lines. These features in the stellar continuum indicate a lack of young stars, similar to low-redshift post-starburst galaxies whose star formation was recently quenched. We find that the stellar mass (log(M_* / M_sun) > 10.6) of each quasar host grew in a starburst episode at redshift 7 or 8. One of the targets exhibits little ongoing star formation, as evidenced by the photometric signature of the Balmer break and a lack of spatially resolved H-alpha emission, placing it well below the star formation main sequence at z = 6. The other galaxy is transitioning to a quiescent phase; together, the two galaxies represent the most distant massive post-starburst galaxies known. The maturity of these two galaxies is further supported by the stellar velocity dispersions of their host galaxies, placing them slightly above the upper end of the local M_BH - sigma_* relation. The properties of our two post-starburst galaxies, each hosting an active SMBH with log(M_BH / M_sun) > 9, suggests that black holes played a major role in shaping the formation of the first massive galaxies in the Universe.         

</div>

<div id="div_fig1">

<img src="tmp_2409.07113/./fig3_nat_v5.png" alt="Fig4" width="100%"/>

**Figure 4. -** **Growth pathways of the two quasar host galaxies and their SMBHs.**(_Left:_) Stellar mass assembly as a function of cosmic time based on the delayed-$\tau$ SFH models (solid lines) and non-parametric SFH models (dotted lines).
For comparison, the average growth curve from the \textsf{Trinity} simulation (Zhang et al. 2023 \cite{Zhang24}) is shown in green, and that based on the dark matter halo merger history from Scoggins \& Haiman (2023) \cite{Scoggins24} is shown in cyan.
 Both model curves are fine-tuned to match the observed stellar mass of $\targetone$.
(_Middle:_) BH mass assembly as a function of time.  The red line show cases in which $\targetone$ continuously accretes at the Eddington limit (solid) or at the observed Eddington ratio ($10\%$; dashed) both with $10\%$ radiative efficiency.
The blue lines show the models of ref \cite{Scoggins24}, where the seed BH of $\targetone$ is a light seed of $50 M_\odot$(solid line), or a heavy seed of  $10^5 M_\odot$(dashed line).
The four grey lines show the semi-analytical model (SAM) from Li et al. (2023) \cite{LiW23},  where seed BHs grow by changing Eddington ratios (including super-Eddington accretion rates) for short periods of time.
The four BHs presented here are a representative sample of BHs that reach $\sim 10^9 M_\odot$ by $z=6$.
These model growth curves are scaled to match the BH mass of $\targetone$.
(_Right:_) Evolution in the $M_\mathrm{BH}$ -- $M_*$ plane (solid line: delayed-$\tau$ SFH, dotted line: non-parametric SFH model).
Luminous quasars whose host stellar emission is detected in JWST \cite{Stone23, Stone24, Yue24}(pink) and faint  AGN from the JWST JADES program \cite{Maiolino23b}(orange)  are shown for comparison.
ALMA dynamical masses and Mg{\sc ii}-based $M_\mathrm{BH}$ of quasar hosts compiled in ref \cite{Izumi21b} are shown as grey crosses.
The $z=4.7$ quiescent galaxy GS-9209 \cite{Carnall23} is shown in green.
The stellar and BH masses of the two $z>6$ quasar host galaxies in the earlier study of ref \cite{Ding23}, which includes $\targetone$, are shown as open red stars.
The local bulge mass -- BH mass relation from ref \cite{KorHo13} is shown as the grey line.
 (*fig:J2236t*)

</div>
<div id="div_fig2">

<img src="tmp_2409.07113/./fig1_nat1_v5.png" alt="Fig1.1" width="50%"/><img src="tmp_2409.07113/./fig1_nat2_v5.png" alt="Fig1.2" width="50%"/>

**Figure 1. -** **JWST NIRSpec G395M spectra** of $\targetone$ and $\targettwo$ with Balmer absorption lines seen in each, thus revealing their post-starburst nature.
(_Top:_) The decomposed quasar and host galaxy components are shown in blue (dotted: power-law continuum; solid: continuum plus emission line model) and red, respectively.
The grey solid line represents the error spectrum.
Pixels affected by cosmic rays are masked.
(_Bottom left:_) Best-fit galaxy SED models from \textsf{Bagpipes} according to the delayed-$\tau$ SFH model (dark blue).
The extracted host galaxy spectrum is shown in cyan.
The grey bar at the bottom shows the wavelength range used in the SED fit.
NIRCam photometry is shown with the red symbols with the filter transmission curves displayed at the bottom.
The inset panel zooms into the Balmer absorption lines.
(_Bottom right:_) The recovered SFH of the host galaxy (delayed-$\tau$: orange, non-parametric: blue).
The solid lines are the 50 percentiles of the posterior, while the darker (lighter) shaded regions show the 16--84 (2--98) percentile intervals.
For each SFH model, the inferred formation redshift $z_\mathrm{form}$ and quenching redshift $z_\mathrm{quench}$ are indicated by filled and open symbols, respectively.
 (*fig:spec*)

</div>
<div id="div_fig3">

<img src="tmp_2409.07113/./fig4_nat_v3.png" alt="Fig5" width="100%"/>

**Figure 5. -** **Stellar velocity dispersion measurements.**(_Left:_)  The  relationship between $M_\mathrm{BH}$ and stellar velocity dispersion $\sigma_*$.
 Symbols and colors are the same as in Figure \ref{fig:J2236t}.
Note that $\sigma_*$ measurements are based on stellar absorption lines for our targets and GS-9209, while the JADES samples use [O {\sc iii}]  emission line profiles to estimate $\sigma_*$.
For comparison, the $z<1$ quasars from the SDSS Reverberation Mapping project \citep{Matsuoka15} are shown as cyan dots.
The grey line and shading show the local relation \cite{KorHo13} and its scatter.
(_Right:_) The comparison of  $\sigma_*$ measured from JWST spectroscopic data and values estimated from the width of the [O {\sc iii}] emission line.
The latter follows the procedure described in ref \cite{Uebler23}.
The diagonal dashed line is the one-to-one relation, while the dotted lines indicate the case where the [O {\sc iii}]-based $\sigma_*$ is offset from the spectroscopic $\sigma_*$ by $+0.1$ and $+0.2$ dex, respectively.
  (*fig:Msigma*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.07113"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

180  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
